This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [ ]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'UYQHo6v_BzV29R4-xd5S'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [18]:
# First, import the relevant modules
import requests
import json

In [16]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

#Proper quandl URL
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?rows=1&api_key=UYQHo6v_BzV29R4-xd5S'

#attempt to grab
oneday = requests.get(url)


In [45]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
jdat = oneday.json()

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [30]:
# 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017

# Define the URL
url1 = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=UYQHo6v_BzV29R4-xd5S'

# Send the request
year_get = requests.get(url1)



In [31]:
# 2. Convert the returned JSON object into a Python dictionary.
year_dat = year_get.json()

In [153]:
# 3. Calculate what the highest and lowest opening prices were for the stock in this period.

# Pull column names and data out of the dictionary
col_names = year_dat['dataset_data']['column_names']
col_dat = year_dat['dataset_data']['data']

# Create a new dictionary from those columns
data = dict.fromkeys(col_names, [])

In [154]:
#still working on 3 here, but doing it this way so I can reset the dict in case it messes up
#I don't even understand the point of using a dictionary if this is what I have to do, but, I digress
# Splitting the values from each part of the data to store them properly in the dictionary.
date = []
for val in col_dat:
    date.append(val[0])
data['Date'] = date

op = []
for val in col_dat:
    if val[1] == None:
        op.append(0)
    else:
        op.append(val[1])
data['Open'] = op

hi = []
for val in col_dat:
    hi.append(val[2])
data['High'] = hi

lo = []
for val in col_dat:
    lo.append(val[3])
data['Low'] = lo

clo = []
for val in col_dat:
    clo.append(val[4])
data['Close'] = clo

ch = []
for val in col_dat:
    ch.append(val[5])
data['Change'] = ch

tv = []
for val in col_dat:
    tv.append(val[6])
data['Traded Volume'] = tv

to = []
for val in col_dat:
    to.append(val[7])
data['Turnover'] = to

lp = []
for val in col_dat:
    lp.append(val[8])
data['Last Price of the Day'] = lp

dtu = []
for val in col_dat:
    dtu.append(val[9])
data['Daily Traded Units'] = dtu

dto = []
for val in col_dat:
    dto.append(val[10])
data['Daily Turnover'] = dto

In [159]:
#Finally, actually answering three by finding the maximimum and minimim values
print('Highest opening price: ' + str(max(data['Open'])))

# to get the proper minimum opening price, I will have to do something else, when parsing the opening prices, I replaced None 
# with 0, as that was the easiest way to think of without using numpy. The min will have to come from a dataset where None isn't 
#replaced with 0

op2 = []
for val in op:
    if val > 0:
        op2.append(val)

print('Lowest opening price: ' + str(min(op2)))

Highest opening price: 53.11
Lowest opening price: 34.0


In [156]:
# 4. What was the largest change in any one day (based on High and Low price)?

#Write a loop which calculates the differencecs between high and low prices based on 
diffs = []
for n in hi:
    for i in lo:
        diff = n-i
        diffs.append(diff)
print('Largest positive change: ' + str(max(diffs)))
print('Largest negative change: ' + str(min(diffs)))
    

Largest positive change: 19.92
Largest negative change: -18.36


In [160]:
# 5. What was the largest change between any two days (based on Closing Price)?

# The original data was returned in reverse chronological order, so difference in days can be calculated like so:
# day(x) - day before(y), this is done with a list comprehension.
clo_diffs = [x-y for x, y in zip(clo, clo[1:])]

#retun the largest changes
print('Largest positive daily change: ' + str(max(clo_diffs)))
print('Largest negative daily change: ' + str(min(clo_diffs)))

Largest positive daily change: 1.7199999999999989
Largest negative daily change: -2.559999999999995


In [158]:
# 6. What was the average daily trading volume during this year?

# calculate the average trade volume by sum of n/n(the normal way to calculate average) 
print(sum(tv)/len(tv))



89124.33725490196
